#### **Ingesta del archivo "genre.csv"**

In [0]:
dbutils.widgets.text("environment","production")
var_environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("file_date","2024-12-16")
var_file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Librerías

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
from pyspark.sql.functions import current_timestamp,lit

##### Esquema

In [0]:
df_genre_schema = StructType([
    StructField("genreId",IntegerType(),False),
    StructField("genreName",StringType(),True)
])

##### Leer dataframe

In [0]:
df_genre = spark.read.format("csv").option("header","true").schema(df_genre_schema).load(f"{bronze_folder_path}/{var_file_date}/genre.csv")

##### Renombrar columnas

In [0]:
df_genre_renamed = df_genre.withColumnsRenamed({"genreId":"genre_id","genreName":"genre_name"})

##### Añadir columnas

In [0]:
df_genre_final = add_ingestion_date(df_genre_renamed).withColumn("environment",lit(var_environment)).withColumn("file_date",lit(var_file_date))

##### Escribir dataframe es Silver

In [0]:
#df_genre_final.write.mode("overwrite").parquet(f"{silver_folder_path}/genres")
df_genre_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genres")

In [0]:
dbutils.notebook.exit("Success")